In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import time
import numpy as np
from functools import partial
import itertools


In [3]:
import pydrake
import meshcat

from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import SceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial
from pydrake.all import Variable, Expression, RotationMatrix
from pydrake.all import MultibodyPositionToGeometryPose, ConnectMeshcatVisualizer, Role, Sphere
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere, VPolytope,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff, InverseKinematics
)

import pydrake.symbolic as sym
import symbolic_parsing_helpers as symHelpers

from meshcat import Visualizer

import mcubes

import ipywidgets as widgets
from IPython.display import display 

# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])

# Sporadically need to run `pkill -f meshcat`

from symbolic_parsing_helpers import generate_rationalize_trig_expr_rules
from symbolic_parsing_helpers import xreplace
from symbolic_parsing_helpers import NotRationalFunctionException
# from symbolic_parsing_helpers import rationalize_trig_expr
from pydrake.all import RationalFunction
snopt = SnoptSolver()
def xreplace(expr, rules):
        if isinstance(expr, float) or isinstance(expr, sym.Variable):
            return expr
        assert isinstance(expr, sym.Expression), expr
        for old, new in rules:
            if expr.EqualTo(old):
                return new
        ctor, old_args = expr.Deconstruct()
        new_args = [xreplace(e, rules) for e in old_args]
        return ctor(*new_args)

def MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)
def MakeFromVPolytopeOrEllipseSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    print(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return VPolytope(query, geom, expressed_in)


from pydrake.all import RationalForwardKinematics, FindBodyInTheMiddleOfChain

In [4]:
# prog = MathematicalProgram()

# x = [sym.Variable("x{}".format(i)) for i in range(3)]
# prog.AddIndeterminates(x)
# poly, Q = prog.NewSosPolynomial(sym.Variables(x),2)
# poly.SetIndeterminates(sym.Variables(x))
# print(prog)
# prog.AddSosConstraint(poly)
# print(prog)

In [5]:
x = sym.Variable('x')
r1 = RationalFunction(sym.Polynomial(2*x+1), sym.Polynomial(x**2-1))
r2 = RationalFunction(sym.Polynomial(2*x+1), sym.Polynomial(x**2-1))
r3 = RationalFunction(sym.Polynomial(2*x+1), sym.Polynomial(x+1)*sym.Polynomial(x-1))
r4 = RationalFunction(sym.Polynomial(2*x+1), sym.Polynomial(x+1))

print(r1+r2)
print(r1+r3)
print(r1+r4)

(2*1 + 4*x) / (-1*1 + 1*x^2)
(2*1 + 4*x) / (-1*1 + 1*x^2)
(1*x + 3*x^2 + 2*x^3) / (-1*1 + -1*x + 1*x^2 + 1*x^3)


# Build simulation Environment

In [6]:
simple_collision = True
# gripper_welded = True

vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()

display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)


q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
slider_context = plant.CreateDefaultContext()
diagram.Publish(context)
query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))


    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)


inspector = query.inspector()
pairs = inspector.GetCollisionCandidates()


geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
HPolyhedronSets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
VPolyhedronSets = {geom:MakeFromVPolytopeOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}



body_indexes_by_geom_id = {geom:
                           plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
<GeometryId value=71>
<GeometryId value=79>
<GeometryId value=113>
<GeometryId value=115>
<GeometryId value=117>
<GeometryId value=85>
<GeometryId value=91>


# Construct Rational Kinematics

In [7]:
forward_kin = RationalForwardKinematics(plant)
q_star = np.zeros(forward_kin.t().shape[0])
t_kin = forward_kin.t()
def convert_RationalForwardPoseList_to_TransformExpressionList(pose_list):
    ret = []
    for p in pose_list:
        ret.append(p.asRigidTransformExpr())
    return ret

def convert_t_to_q(t):
    q =np.arctan2(2*t/(1+t**2), (1-t**2)/(1+t**2))
    return q

def convert_q_to_t(t):
    q =np.tan(t/2)
    return q

def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)


def eval_cons_rational(t0, t1, t2, c, tol):
        q = convert_t_to_q(np.array([t0, t1, t2])) 
        return 1-1*float(c.evaluator().CheckSatisfied(q, tol))

col_func_handle_rational = partial(eval_cons_rational, c=collision_constraint, tol=0.01)
link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
link_poses_by_body_index_multilinear_pose = forward_kin.CalcLinkPosesAsMultilinearPolynomials(q_star, 
                                                         plant.world_body().index())

X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
X_WA_multilinear_list = [(r.rotation().copy(), r.translation().copy()) for r in link_poses_by_body_index_multilinear_pose]
geomA, geomB = next(iter(pairs))
index = FindBodyInTheMiddleOfChain(plant, body_indexes_by_geom_id[geomA], body_indexes_by_geom_id[geomB])
print(index)

t_space_vertex_position_by_geom_id ={}
for geom in geom_ids:
    VPoly = VPolyhedronSets[geom]
    num_verts = VPoly.vertices().shape[1]
    X_WA = X_WA_list[int(body_indexes_by_geom_id[geom])]
    R_WA = X_WA.rotation().matrix()
    p_WA = X_WA.translation()
    vert_pos = R_WA@(VPoly.vertices())+ np.repeat(p_WA[:, np.newaxis],num_verts,1)
    t_space_vertex_position_by_geom_id[geom] = vert_pos
    
t_space_vertex_position_by_geom_id2 ={}
for geom in geom_ids:
    VPoly = VPolyhedronSets[geom]
    num_verts = VPoly.vertices().shape[1]
    R_WA, p_WA = X_WA_multilinear_list[int(body_indexes_by_geom_id[geom])]

    vert_pos = R_WA@(VPoly.vertices())+ np.repeat(p_WA[:, np.newaxis],num_verts,1)
    with np.nditer(vert_pos, op_flags=['readwrite'], flags=["refs_ok"]) as it:
        for x in it:
            x[...] = forward_kin.ConvertMultilinearPolynomialToRationalFunction(x.item())
        
    t_space_vertex_position_by_geom_id2[geom] = vert_pos


BodyIndex(4)


In [8]:
q_low = plant.GetPositionLowerLimits()
q_high = plant.GetPositionUpperLimits()
t_low = forward_kin.ComputeTValue(q_low,q_star)
t_high = forward_kin.ComputeTValue(q_high,q_star)
print(t_low)
print(t_high)


[-1.73205081 -1.73205081 -1.73205081]
[1.73205081 1.73205081 1.73205081]


# Sliders

In [9]:




def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))

col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

def eval_cons_rational(t0, t1, t2, c, tol):
    q = convert_t_to_q(np.array([t0, t1, t2]).reshape(1,-1)).squeeze() 
    return col_func_handle(*q)
   
col_func_handle_rational = partial(eval_cons_rational, c=collision_constraint, tol=0.01)

def showres(q):
#     set_joint_ang(q[0],0)
#     set_joint_ang(q[1],1)
#     set_joint_ang(q[2],2)
    plant.SetPositions(slider_context, q)
    col = col_func_handle(*q)
    t = convert_q_to_t(np.array(q).reshape(1,-1)).squeeze()
    if col:
        vis2["t"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
        vis2["t"].set_transform(
                meshcat.transformations.translation_matrix(t))
    else:
        vis2["t"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x3EFF00))
        vis2["t"].set_transform(
                meshcat.transformations.translation_matrix(t))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col , end = "\r")
    
def showres_t(t):
    q = convert_t_to_q(t)
    showres(q)

sliders = []
sliders.append(widgets.FloatSlider(min=t_low[0], max=t_high[0], value=0, description='t0'))
sliders.append(widgets.FloatSlider(min=t_low[1], max=t_high[1], value=0, description='t1'))
sliders.append(widgets.FloatSlider(min=t_low[2], max=t_high[2], value=0, description='t2'))

q = q0.copy()
t = convert_q_to_t(np.array(q))
def handle_slider_change(change, idx):
    t[idx] = change['new']
    print(q, end="\r")
    showres_t(t)
    plane_callback(t)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

In [10]:
for slider in sliders:
    display(slider)

display(vis.jupyter_cell())
display(vis2.jupyter_cell())


FloatSlider(value=0.0, description='t0', max=1.7320508075624863, min=-1.7320508075624863)

FloatSlider(value=0.0, description='t1', max=1.7320508075624863, min=-1.7320508075624863)

FloatSlider(value=0.0, description='t2', max=1.7320508075624863, min=-1.7320508075624863)

In [11]:
def visualize_trajectory(traj):
    builder = DiagramBuilder()

    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(time_step=0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    parser = Parser(plant)
    parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
                "drake/manipulation/models/wsg_50_description/package.xml")))

    directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
        if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)

    q0 = [-0.2, -1.2, 1.6]
    index = 0
    for joint_index in plant.GetJointIndices(models[0].model_instance):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[index])
            index += 1

    plant.Finalize()

    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))

    traj_system = builder.AddSystem(TrajectorySource(traj))
    builder.Connect(traj_system.get_output_port(), to_pose.get_input_port())

    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

    vis_diagram = builder.Build()

    simulator = Simulator(vis_diagram)
    meshcat.start_recording()
    simulator.AdvanceTo(traj.end_time())
    meshcat.publish_recording()
    with open("/tmp/spp_shelves.html", "w") as f:
        f.write(meshcat.vis.static_html())


## Marching Cubes Obstacles

In [12]:
N = 50
vertices, triangles = mcubes.marching_cubes_func(tuple(t_low), tuple(t_high), N, N, N, col_func_handle_rational, 0.5)
vis2["collision_constraint"].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=True))
# q = q0.copy()
# showres(q)

# Define Iris In T-Space

In [13]:
#  Now IRIS in configuration space, using dReal to solve for the growth volume
# through the nonconvex kinematics.

from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)


def TangentPlane(self, point):
    a = 2 * self.A().T @ self.A() @ (point - self.center())
    a = a / np.linalg.norm(a)
    b = a.dot(point)
    return a, b

Hyperellipsoid.TangentPlane = TangentPlane


query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))

sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()

# For SNOPT test.
snopt = SnoptSolver()



In [14]:
def GrowthVolumeRational(E, X_WA, X_WB, setA, setB, A, b, guess=None):
    snopt = SnoptSolver()
    prog = MathematicalProgram()
    t_kin = forward_kin.t()
    prog.AddDecisionVariables(t_kin)
    
    if guess is not None:
        prog.SetInitialGuess(t_kin, guess)

    prog.AddLinearConstraint(A, b-np.inf, b, t_kin)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), t_kin)

    p_WA = X_WA.multiply(p_AA+0)
    p_WB = X_WB.multiply(p_BB+0)

    prog.AddConstraint(eq(p_WA, p_WB))
    result = snopt.Solve(prog)
    return result.is_success(), result.get_optimal_cost(), result.GetSolution(t_kin)


In [15]:

def iris_rational_space(query, point, require_containment_points=[], 
                        termination_threshold=2e-2, iteration_limit=100):
    dReal_polytope_tol = 1e-2
    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    rational_lb = forward_kin.ComputeTValue(lb, q_star)
    ub = plant.GetPositionUpperLimits()
    rational_ub = forward_kin.ComputeTValue(ub, q_star)
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere
    
    link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
    X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(rational_lb, rational_ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromHPolyhedronOrEllipseSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
    body_indexes_by_geom_id = {geom:
                               plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 
    
    #Turn onto true to certify regions using SOS at each iteration.
    certify = False
    # refine polytopes if not certified collision free
    refine = False and certify
        
    iteration = 0
    num_faces = 2*len(lb)
    while True:
        ## Find separating hyperplanes

        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
                X_WB = X_WA_list[int(body_indexes_by_geom_id[geomB])]
                success, growth, qstar = GrowthVolumeRational(E,
                    X_WA, X_WB,
                    sets[geomA], sets[geomB], 
                    A[:num_faces,:], b[:num_faces] - dReal_polytope_tol, 
                    point)
                qstar = qstar.squeeze()
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if certify:
                pass
            

        if any([np.any(A[:num_faces,:] @ t > b[:num_faces]) for t in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])

        E = P.MaximumVolumeInscribedEllipsoid()
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P

In [16]:
seed_points = np.tan(np.array([[0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]])    # out of shelf 1
                        /2)
for i in range(seed_points.shape[0]):
    vis2['seedpoints']["seedpoint"+str(i)].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
    vis2['seedpoints']["seedpoint"+str(i)].set_transform(
                meshcat.transformations.translation_matrix(seed_points[i,:]))

# traj = PiecewisePolynomial.FirstOrderHold(np.array([0, 1]), np.array([seed_points[4], seed_points[1]]).T)
# visualize_trajectory(traj)

In [17]:

regions = []
ellipses = []
for i in range(seed_points.shape[0]):
    
    start_time = time.time()
    hpoly = iris_rational_space(query, seed_points[i,:], require_containment_points=[seed_points[i,:]], iteration_limit=100)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    regions.append(hpoly)
    ellipses.append(ellipse)

print("SUCCESS")

geomA=iiwa::link7, geomB=shelves::top
snopt_example=[ 0.25686152 -0.44650254  1.37305101], growth = 138.27790339845825
geomA=iiwa::link7, geomB=wsg::right_collision
geomA=wsg::left_collision, geomB=wsg::right_collision
geomA=wsg::right_collision, geomB=shelves::shelf_upper
snopt_example=[ 0.14471298 -0.66482365  0.73909564], growth = 147.06126401711242
snopt_example=[ 0.26684134 -0.53520202  0.93115289], growth = 151.49619658550938
geomA=wsg::body_collision, geomB=shelves::top
geomA=wsg::body_collision, geomB=shelves::shelf_upper
snopt_example=[ 0.32551918 -0.53379534  1.10297422], growth = 138.142516784699
geomA=wsg::left_collision, geomB=shelves::top
geomA=wsg::right_collision, geomB=shelves::top
geomA=iiwa::link7, geomB=shelves::shelf_lower
snopt_example=[ 0.5119587  -0.80097613  1.4041356 ], growth = 88.94239682323666
geomA=wsg::body_collision, geomB=shelves::shelf_lower
geomA=wsg::left_collision, geomB=shelves::shelf_lower
snopt_example=[ 0.27896158 -0.8160305   0.60048261], growt

snopt_example=[ 0.19186344 -1.23672615 -0.35546054], growth = 9.023992444050037
geomA=wsg::body_collision, geomB=shelves::top
snopt_example=[ 0.03898991 -1.0702109  -0.17559755], growth = 4.176028989953181
geomA=wsg::body_collision, geomB=shelves::shelf_upper
snopt_example=[ 0.14420559 -1.25506828 -0.25494605], growth = 3.585332366144627
geomA=wsg::left_collision, geomB=shelves::top
snopt_example=[-0.04434811 -1.22664273 -0.13630201], growth = 0.8715330056431299
snopt_example=[-0.01399985 -0.99501865  0.0385244 ], growth = 9.19832182806266
geomA=wsg::right_collision, geomB=shelves::top
geomA=iiwa::link7, geomB=shelves::shelf_lower
geomA=wsg::body_collision, geomB=shelves::shelf_lower
geomA=wsg::left_collision, geomB=shelves::shelf_lower
geomA=wsg::right_collision, geomB=shelves::shelf_lower
geomA=iiwa::link7, geomB=wsg::left_collision
geomA=iiwa::link7, geomB=shelves::shelf_upper
geomA=wsg::left_collision, geomB=shelves::shelf_upper
0
geomA=iiwa::link7, geomB=shelves::top
geomA=iiwa::l

# Choose poly to cert

In [18]:
poly_to_cert_safe = regions[2]
Atmp = poly_to_cert_safe.A()
btmp = poly_to_cert_safe.b()
poly_to_cert_safe = HPolyhedron(Atmp, btmp-0.3)


poly_to_cert_unsafe = regions[0]
Atmp = poly_to_cert_unsafe.A()
btmp = poly_to_cert_unsafe.b()
poly_to_cert_unsafe = HPolyhedron(Atmp, btmp+0.3)



## Plot Regions

In [19]:
def inpolycheck(q0,q1,q2, A, b):
    q = np.array([q0, q1, q2])
    res = np.min(1.0*(A@q-b<=0))
    #print(res)
    return res
eps = 0.5
meshes = []
col1 = 0x002BFF
col2 = 0x3EFF00 
idx = 0
step = 1/(1.0*len(regions))
R = np.array([[1, 0, 0],[0, 0, 1],[0,1, 0]])
order = (0,1,2)
for i, region in enumerate(regions):
    A = region.A()[:,order]
    b = region.b()

    col_hand = partial(inpolycheck, A=A, b=b)
    vertices, triangles = mcubes.marching_cubes_func(tuple(t_low-eps), tuple(t_high+eps), 50, 50, 50, col_hand, 0.5)

    regstr = "regions" +str(idx)
    ellstr = "ellipse" +str(idx)
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    vis2['regions'][regstr].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)


    
    idx+=1
A_poly = poly_to_cert_safe.A()         
b_poly = poly_to_cert_safe.b() 
col_hand = partial(inpolycheck, A=A_poly, b=b_poly)
Nhere = 75
vertices, triangles = mcubes.marching_cubes_func(tuple(t_low-eps), tuple(t_high+eps), Nhere, Nhere, Nhere, col_hand, 0.5)
mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
mat.opacity = 0.3
vis2['poly_to_cert_safe'].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)
    
A_poly = poly_to_cert_unsafe.A()         
b_poly = poly_to_cert_unsafe.b() 
col_hand = partial(inpolycheck, A=A_poly, b=b_poly)
vertices, triangles = mcubes.marching_cubes_func(tuple(t_low), tuple(t_high), Nhere, Nhere, Nhere, col_hand, 0.5)
mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
mat.opacity = 0.3
vis2['poly_to_cert_unsafe'].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)

In [20]:
#monomial generation helpers
from pydrake.all import GenerateMonomialBasisOrderAllUpToOneExceptOneUpToTwo, GenerateMonomialBasisWithOrderUpToOne
print(GenerateMonomialBasisOrderAllUpToOneExceptOneUpToTwo(sym.Variables(t_kin)))
print(GenerateMonomialBasisWithOrderUpToOne(sym.Variables(t_kin)))


[<Monomial "1"> <Monomial "t[0]"> <Monomial "t[1]">
 <Monomial "t[0] * t[1]"> <Monomial "t[2]"> <Monomial "t[0] * t[2]">
 <Monomial "t[1] * t[2]"> <Monomial "t[0] * t[1] * t[2]">
 <Monomial "t[0]^2"> <Monomial "t[0]^2 * t[1]"> <Monomial "t[0]^2 * t[2]">
 <Monomial "t[0]^2 * t[1] * t[2]"> <Monomial "t[1]^2">
 <Monomial "t[0] * t[1]^2"> <Monomial "t[1]^2 * t[2]">
 <Monomial "t[0] * t[1]^2 * t[2]"> <Monomial "t[2]^2">
 <Monomial "t[0] * t[2]^2"> <Monomial "t[1] * t[2]^2">
 <Monomial "t[0] * t[1] * t[2]^2">]
[<Monomial "1"> <Monomial "t[0]"> <Monomial "t[1]">
 <Monomial "t[0] * t[1]"> <Monomial "t[2]"> <Monomial "t[0] * t[2]">
 <Monomial "t[1] * t[2]"> <Monomial "t[0] * t[1] * t[2]">]


# Certify the Regions

In [21]:
def construct_first_order_separating_hyperplane(prog, t, order = 2, plane_name = ''):
    if plane_name != '':
        plane_name = '_'+plane_name
    t_basis = sym.MonomialBasis(t, order)
    t_basis = np.array([sym.Polynomial(v) for v in t_basis])
    a_A_coeffs = prog.NewContinuousVariables(3, t_basis.shape[0], 'a_A'+plane_name)
    a_poly = a_A_coeffs@t_basis
    
    b_A_coeffs = prog.NewContinuousVariables(1, t_basis.shape[0], 'b_A'+plane_name)
    b_poly = b_A_coeffs@t_basis
    
    for i, p in enumerate(a_poly):
        a_poly[i].SetIndeterminates(sym.Variables(t))
    for i, p in enumerate(b_poly):
        b_poly[i].SetIndeterminates(sym.Variables(t))
    dec_vars = [*a_A_coeffs.flatten().tolist(), *b_A_coeffs.flatten().tolist()]
    ntmp = a_A_coeffs.flatten().shape[0]
    Qtmp, btmp = np.eye(ntmp), np.zeros(ntmp)
    prog.AddQuadraticCost(Qtmp, btmp, a_A_coeffs.flatten())
    return prog, a_poly, b_poly.item(), dec_vars


def putinarPsatConstraint(prog, p, t,  
                         poly_to_cert, lagrange_mult_degree = 2, var_epsilon = None):
    A = poly_to_cert.A()
    b = poly_to_cert.b()
    n = b.shape[0]
    if var_epsilon is None:
        var_epsilon = np.zeros(n)
#     lagrange_poly, Q = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
    lagrange_poly, Q = prog.NewSosPolynomial(GenerateMonomialBasisOrderAllUpToOneExceptOneUpToTwo(
        sym.Variables(t)))
    lagrange_poly.SetIndeterminates(sym.Variables(t))
    prog.AddSosConstraint(lagrange_poly)
    constraint_poly = lagrange_poly
    for i in range(n):
#         lagrange_poly, Q = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
        lagrange_poly, Q = prog.NewSosPolynomial(
            GenerateMonomialBasisOrderAllUpToOneExceptOneUpToTwo(sym.Variables(t)))
        lagrange_poly.SetIndeterminates(sym.Variables(t))
        prog.AddSosConstraint(lagrange_poly)
        constraint_poly += lagrange_poly*sym.Polynomial(b[i]-var_epsilon[i]-A[i,:]@t)
    constraint_poly.SetIndeterminates(sym.Variables(t))
    tol = 1e-3
    prog.AddEqualityConstraintBetweenPolynomials(constraint_poly, p-tol)
#     prog.AddSosConstraint(p-constraint_poly)
    return prog, (constraint_poly, p-tol)


def makeBodyHyperplaneSidePolynomials(prog, a_plane_poly, b_plane_poly,
                          VPoly, R_WA, p_WA, t, poly_to_cert, leq_or_geq,
                          lagrange_mult_degree = 2, var_epsilon = None, base_point_as_multilinear_poly = None):
    num_verts = VPoly.vertices().shape[1]

    vertex_pos = R_WA@(VPoly.vertices())+ np.repeat(p_WA[:, np.newaxis],num_verts,1)
    zero_poly = sym.Polynomial(0)
    if base_point_as_multilinear_poly is None:
        base_point_as_multilinear_poly = np.array([zero_poly for i in range(3)])
    
    centered_pos = vertex_pos #- np.repeat(base_point_as_multilinear_poly[:, np.newaxis],num_verts,1)
    
    dens = np.ndarray(shape = vertex_pos.shape,dtype = object)
    nums = np.ndarray(shape = vertex_pos.shape,dtype = object)
    for i, row in enumerate(vertex_pos):
        
        for j, v in enumerate(row):
            vertex_pos[i,j] = forward_kin.ConvertMultilinearPolynomialToRationalFunction(v)
            dens[i,j] = vertex_pos[i,j].denominator()
            nums[i,j] = vertex_pos[i,j].numerator()

    for c in range(dens.shape[1]):
        for r in range(dens.shape[0]-1):
            if not dens[r,c].EqualTo(dens[r+1,c]):
                raise ValueError("problem")
            
#     plane_rats = a_plane_poly.dot(vertex_pos)
    plane_polys = np.array([None for _ in range(vertex_pos.shape[1])])
    prog_eps = 1e-12
    for i in range(vertex_pos.shape[1]):
#         p.SetIndeterminates(sym.Variables(t))
        
        if leq_or_geq == 'leq':
            #a^Tx + b <= -1 -> -(a^Tx+b+1)>=0
            plane_polys[i] = (-a_plane_poly.dot(nums[:,i])-(b_plane_poly+1)*(dens[:,i].sum()))
            # (b_plane_poly-prog_eps)*p.denominator()-p.numerator() 
            # (b_plane_poly-prog_eps)*p.denominator()-p.numerator()
        elif leq_or_geq == 'geq':
            #a^Tx+b >= 1 -> a^Tx+b -1 >= 0
            plane_polys[i] = (a_plane_poly.dot(nums[:,i])+(b_plane_poly-1)*(dens[:,i].sum()))
        else:
            raise ValueError("leq_or_geq arg must be leq or geq not {}".format(leq_or_geq))  
        plane_polys[i].SetIndeterminates(sym.Variables(t))
        
#     plane_rats = a_plane_poly.dot(vertex_pos)
#     plane_polys2 = np.array([None for _ in plane_rats])
#     prog_eps = 1e-12
#     for i, p in enumerate(plane_rats):
#         p.SetIndeterminates(sym.Variables(t))

#         if leq_or_geq == 'leq':
#             #a^Tx + b <= -1 -> -(a^Tx+b+1)>=0
#             plane_polys2[i] = (-(p.numerator()+(b_plane_poly+1)*p.denominator())).item()
#             # (b_plane_poly-prog_eps)*p.denominator()-p.numerator() 
#             # (b_plane_poly-prog_eps)*p.denominator()-p.numerator()
#         elif leq_or_geq == 'geq':
#             #a^Tx+b >= 1 -> a^Tx+b -1 >= 0
#             plane_polys2[i] = (p.numerator()+(b_plane_poly-1)*p.denominator()).item()
#         else:
#             raise ValueError("leq_or_geq arg must be leq or geq not {}".format(leq_or_geq))  
#     for (p,q) in zip(plane_polys, plane_polys2):
#         print(p.EqualTo(q))
    return plane_polys


def add_pair_constraint(geomA, geomB, prog, poly_to_cert, 
                        lagrange_mult_degree = 2, var_epsilon = None):
    VPolyA, VPolyB = VPolyhedronSets[geomA], VPolyhedronSets[geomB]
    prog, a_plane_poly, b_plane_poly, dec_vars = construct_first_order_separating_hyperplane(prog, t_kin)
        
    R_WA, p_WA = X_WA_multilinear_list[int(body_indexes_by_geom_id[geomA])]
    R_WB, p_WB  = X_WA_multilinear_list[int(body_indexes_by_geom_id[geomB])]
    
    base_point_poly = None#R_WA@VPolyA.vertices().mean(axis = 1)-p_WA
#     base_point = np.array([forward_kin.ConvertMultilinearPolynomialToRationalFunction(p) for p in base_point_poly])
    
    
    plane_polys_A = makeBodyHyperplaneSidePolynomials(prog, a_plane_poly, b_plane_poly, 
                                                     VPolyA, R_WA, p_WA , t_kin, 
                                                     poly_to_cert, 'leq', lagrange_mult_degree,
                                                     var_epsilon, base_point_poly)
    plane_polys_B = makeBodyHyperplaneSidePolynomials(prog, a_plane_poly, b_plane_poly,
                                                     VPolyB, R_WB, p_WB , t_kin, 
                                                     poly_to_cert, 'geq', lagrange_mult_degree,
                                                     var_epsilon, base_point_poly)
    plane_polys = np.array(plane_polys_A.tolist()+ plane_polys_B.tolist()).squeeze()
    
    
    s_prod_pairs = []
    
    for p in plane_polys:
        prog, (constraint_poly, p_with_tol) = putinarPsatConstraint(prog, p, t_kin,  
                         poly_to_cert, lagrange_mult_degree = lagrange_mult_degree, var_epsilon = var_epsilon)
        s_prod_pairs.append((constraint_poly, p_with_tol))
    return prog, plane_polys, (a_plane_poly, b_plane_poly), dec_vars, s_prod_pairs


def construct_region_safety_problem(poly_to_cert, lagrange_mult_degree = 2, var_epsilon = None, check_archimedean = True, plane_name = ""):
    
    if check_archimedean:
        is_Archimedean,_ = certify_archimedean(poly_to_cert, t_kin)
        if not is_Archimedean:
            raise ValueError("region is not archimedean")
    prog = MathematicalProgram()
    prog.AddIndeterminates(t_kin)
    plane_pairs = {}
    plane_polys_list = []
    s_prod_pairs = {}
    dec_vars = []
    for i, (geomA, geomB) in enumerate(pairs):
        print("pair {}/{}".format(i+1, len(pairs)))
        prog, plane_polys, (a_poly, b_poly), dec_vars0, s_prod_pairs0 = add_pair_constraint(geomA, geomB,
                                                       prog, poly_to_cert,
                                                       lagrange_mult_degree = lagrange_mult_degree,
                                                       var_epsilon = var_epsilon
                                                      )
        plane_polys_list.append(plane_polys)
        plane_pairs[(geomA, geomB)] = (a_poly, b_poly)
        s_prod_pairs[(geomA, geomB)] = s_prod_pairs0
        dec_vars += dec_vars0
#         if i >= 0:
#             break
    return prog, plane_pairs, plane_polys_list, np.array(dec_vars), s_prod_pairs

def certify_region(poly_to_cert, lagrange_mult_degree = 4, var_epsilon = None, check_archimedean = True, numeric_tol = 1e-10):
    prog, plane_pairs, plane_polys_list, dec_vars, s_prod_pairs = construct_region_safety_problem(poly_to_cert, 
                                                                          lagrange_mult_degree = lagrange_mult_degree, 
                                                                          var_epsilon = var_epsilon, 
                                                                          check_archimedean = check_archimedean)
    result = convSolver.Solve(prog)
    numerically_cert = result.is_success() and (np.linalg.norm(result.GetSolution(dec_vars))>= numeric_tol)
    print(np.linalg.norm(result.GetSolution(dec_vars)))
    return numerically_cert, result, prog, s_prod_pairs
              
def certify_archimedean(poly_to_cert, t, lagrange_mult_degree = 2):
    #need to fix
    convSolver = MosekSolver()
    prog = MathematicalProgram()
    prog.AddIndeterminates(t)
        
    t_bar = sym.Polynomial(t@t)

    poly = t_bar
    A = poly_to_cert.A()
    b = poly_to_cert.b()
    n = b.shape[0]
    for i in range(n):
        lagrange_poly, Q = prog.NewSosPolynomial(sym.Variables(t), lagrange_mult_degree)
        lagrange_poly.SetIndeterminates(sym.Variables(t))
        prog.AddSosConstraint(lagrange_poly)
        poly += lagrange_poly*sym.Polynomial(b[i] - A[i,:]@t)
    poly.SetIndeterminates(sym.Variables(t))
    prog.AddSosConstraint(poly)
    result = convSolver.Solve(prog)
    return result.is_success(), prog


In [28]:
#safe
prog = MathematicalProgram()
convSolver = MosekSolver()
prog.AddIndeterminates(t_kin)
is_Archimedean,_ = certify_archimedean(poly_to_cert_safe, t_kin)
print("Poly to cert safe is Archimedean {}".format(is_Archimedean))
safe = True
safe_failures = []

max_iter = 1
iter_num = 0

eps = 0
safe = False
plane_pairs = []
while iter_num < max_iter:
    prog, plane_pairs, plane_polys_list, dec_vars, s_prod_pairs = construct_region_safety_problem(poly_to_cert_safe, 
                                    lagrange_mult_degree = 10, 
                                    var_epsilon = eps*np.ones_like(poly_to_cert_safe.b()), 
                                    check_archimedean = True, 
                                    plane_name = "")
    result = convSolver.Solve(prog)
    print("Var_epsilon = {}, safe = {}".format(eps, result.is_success()))
    if result.is_success():
        safe = True
        break
    else:
        eps += 0.1
        iter_num += 1
print("Safe = {}".format(result.is_success()))
print("Plane vars norm {}".format(np.linalg.norm(result.GetSolution(dec_vars))))
    
#     if not result.is_success():
#         safe_failures += (geomA, geomB)
#     safe = safe and result.is_success()
# print(safe)

Poly to cert safe is Archimedean True
pair 1/15
pair 2/15
pair 3/15
pair 4/15
pair 5/15
pair 6/15
pair 7/15
pair 8/15
pair 9/15
pair 10/15
pair 11/15
pair 12/15
pair 13/15
pair 14/15
pair 15/15
Var_epsilon = 0, safe = True
Safe = True
Plane vars norm 608.2080837979806


In [32]:
def extract_planes(plane_pairs):
    resulting_plane_pairs = {}
    
    for k, (a, b) in plane_pairs.items():
        a_list = []
        for ai in a:
            a_list.append(result.GetSolution(ai))
        resulting_plane_pairs[k] = (np.array(a_list), result.GetSolution(b))
    return resulting_plane_pairs

def EvaluatePlanePair(plane_pair, eval_dict):
    a_res = []
    for ai in plane_pair[0]:
        a_res.append(ai.Evaluate(eval_dict))
    return (np.array(a_res), plane_pair[1].Evaluate(eval_dict))

def extract_s_prod_pairs(s_prod_pairs):
    evaled_pairs = {}
    it = 0
    for k, poly_list in s_prod_pairs.items():
        cur_list = []
        for (const, vert) in poly_list:
            constraint_poly_evaled = result.GetSolution(const)
            vert_poly_evaled = result.GetSolution(vert)
            cur_list.append((constraint_poly_evaled, vert_poly_evaled))
        evaled_pairs[k] = cur_list
    return evaled_pairs

cur_t = np.array([0.57,-0.73,0.37])
planes = extract_planes(plane_pairs)
s_prod_pairs_polys = extract_s_prod_pairs(s_prod_pairs)
print(EvaluatePlanePair(planes[next(iter(pairs))], dict(zip(sym.Variables(t_kin), cur_t))))
print(poly_to_cert_unsafe.A()@cur_t <= poly_to_cert_unsafe.b())



(array([ 8.52562723e+00,  1.94887472e-09, -5.42098946e+00]), 0.5672538197053968)
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


In [24]:
print(len(next(iter(s_prod_pairs.values()))))

NameError: name 's_prod_pairs' is not defined

In [25]:
# unsafe
is_Archimedean,_ = certify_archimedean(poly_to_cert_unsafe, t_kin)
print("Poly to cert safe is Archimedean {}".format(is_Archimedean))

# prog = MathematicalProgram()|
# prog.AddIndeterminates(t)
# safe = True
# safe_failures = []
# prog = MathematicalProgram()
# prog.AddIndeterminates(t)
# plane_pairs = []
# plane_polys_list = []
prog, plane_pairs, plane_polys_list, dec_vars, s_prod_pairs = construct_region_safety_problem(poly_to_cert_unsafe, 
                                    lagrange_mult_degree = 10, 
                                    var_epsilon = None, 
                                    check_archimedean = True, 
                                    plane_name = "")
result = convSolver.Solve(prog)
print("Safe = {}".format(result.is_success()))
print("Plane vars norm {}".format(np.linalg.norm(result.GetSolution(dec_vars))))
planes = extract_planes(plane_pairs)
s_prod_pairs_polys = extract_s_prod_pairs(s_prod_pairs)


Poly to cert safe is Archimedean True
pair 1/15
pair 2/15
pair 3/15
pair 4/15
pair 5/15
pair 6/15
pair 7/15
pair 8/15
pair 9/15
pair 10/15
pair 11/15
pair 12/15
pair 13/15
pair 14/15
pair 15/15


NameError: name 'convSolver' is not defined

In [27]:
convSolver = MosekSolver()
result = convSolver.Solve(prog)
print("Safe = {}".format(result.is_success()))

Safe = False


In [ ]:
res = result.GetSolution()
print(np.linalg.norm(res))

In [ ]:
print(np.array(plane_polys_list).shape)

In [ ]:

convSolver = MosekSolver()
is_Archimedean,_ = certify_archimedean(poly_to_cert_unsafe, t_kin)
print("Poly to cert safe is Archimedean {}".format(is_Archimedean))
start_time = time.time()
failures = []
for j,r in enumerate(regions):
    print("Region {}/{}".format(j, len(regions)-1))
    rtmp = HPolyhedron(r.A(), r.b()+0.5)
    numerically_cert, result, prog = certify_region(rtmp, 
                                                    lagrange_mult_degree = 4,
                                                    check_archimedean = True,
                                                    numeric_tol = 1e-10)
    print("Safe: {}".format(numerically_cert))
    if not numerically_cert:
        failures += [r]
end_time = time.time()
print("Certified in {}".format(end_time-start_time))

In [76]:
# certify region is P compact
i, (geomA, geomB) = next(iter(enumerate(pairs)))
R_WA, p_WA = X_WA_multilinear_list[int(body_indexes_by_geom_id[geomA])]
R_WA, p_WA = R_WA.copy(), p_WA.copy()
X_WA_rat_direct = X_WA_list[int(body_indexes_by_geom_id[geomA])]
R_WA_direct, p_WA_direct = X_WA_rat_direct.rotation().matrix(), X_WA_rat_direct.translation()

for i in range(3):
    for j in range(3):
        R_WA[i,j] = forward_kin.ConvertMultilinearPolynomialToRationalFunction(R_WA[i,j])
    p_WA[i] = forward_kin.ConvertMultilinearPolynomialToRationalFunction(p_WA[i])

num_samp = 10000
bad_evals = {}
num_bad = 0
for i in range(num_samp):
    t_vals = np.random.uniform(t_low, t_high, (3,))
    vals_dict = dict(zip(sym.Variables(t_kin), t_vals))
    conv_R = np.zeros_like(R_WA)
    conv_p = np.zeros_like(p_WA)
    direct_R = np.zeros(R_WA.shape).astype(float)
    direct_p = np.zeros(p_WA.shape).astype(float)
    for i in range(3):
        for j in range(3):
            conv_R[i,j] = R_WA[i,j].numerator().Evaluate(vals_dict)/R_WA[i,j].denominator().Evaluate(vals_dict)
            direct_R[i,j] = R_WA_direct[i,j].Evaluate(vals_dict)
            
        conv_p[i] = p_WA[i].numerator().Evaluate(vals_dict)/p_WA[i].denominator().Evaluate(vals_dict)
        direct_p[i] = p_WA_direct[i].Evaluate(vals_dict)

        if not np.allclose(conv_R.astype(float), direct_R.astype(float))  or not np.allclose(conv_p.astype(float), direct_p.astype(float)) :
            bad_evals[t_vals] = ((conv_R, direct_R), (conv_p, direct_p))
            num_bad += 1
            print(num_bad)
print("done")  

done


In [53]:
vals_dict = dict(zip(sym.Variables(t_kin), [0,0,0]))
R_WA_direct.Evaluate(vals_dict)

AttributeError: 'numpy.ndarray' object has no attribute 'Evaluate'

In [36]:
import scipy 
import utils

x = np.linspace(-1,1, 3)
y = np.linspace(-1,1, 3)
verts = []

for idxx in range(len(x)):
    for idxy in range(len(y)):
        verts.append(np.array([x[idxx], y[idxy]]))
       
tri = scipy.spatial.Delaunay(verts)
plane_triangles = tri.simplices
plane_verts = tri.points[:,:]
plane_verts = np.concatenate((plane_verts, 0*plane_verts[:,0].reshape(-1,1)), axis = 1)

def transform(a,b, p1, p2, plane_verts, plane_triangles):
    alpha = (-b-a.T@p1)/(a.T@(p2-p1))
    offset = alpha*(p2-p1) + p1
    z = np.array([0, 0, 1])
    crossprod = np.cross(utils.normalize(a), z)
    if np.linalg.norm(crossprod) <=1e-4:
        R = np.eye(3)
    else:
        ang = np.arcsin(np.linalg.norm(crossprod))
        axis = utils.normalize(crossprod)
        R = utils.get_rotation_matrix(axis, -ang)
        
    verts_tf = (R@plane_verts.T).T + offset
    return verts_tf

def transform_at_t(cur_t, a_poly, b_poly, p1_rat, p2_rat):
    eval_dict = dict(zip(sym.Variables(t_kin), cur_t))
    a,b = EvaluatePlanePair((a_poly, b_poly), eval_dict)
#     print(f"{a}, {b}")
    p1 = np.array([p.Evaluate(eval_dict) for p in p1_rat])
    p2 = np.array([p.Evaluate(eval_dict) for p in p2_rat])
    return transform(a,b, p1, p2, plane_verts, plane_triangles), p1, p2

def transform_plane_geom_id(geomA, geomB, planes_dict, cur_t):
    vA = t_space_vertex_position_by_geom_id[geomA][:,0]
    vB = t_space_vertex_position_by_geom_id[geomB][:,0]
    a_poly, b_poly = planes_dict[(geomA, geomB)]
    return transform_at_t(cur_t, a_poly, b_poly, vA, vB)

def plot_plane_geom_id(geomA, geomB, planes_dict, cur_t):
    verts_tf, p1, p2 = transform_plane_geom_id(geomA, geomB, planes_dict, cur_t)
    
    #other rat
    eval_dict = dict(zip(sym.Variables(t_kin), cur_t))
    vA2 = t_space_vertex_position_by_geom_id2[geomA][:,1]
    vB2 = t_space_vertex_position_by_geom_id2[geomB][:,1]
    p1_other = np.array([p.Evaluate(eval_dict) for p in vA2])
    p2_other = np.array([p.Evaluate(eval_dict) for p in vB2])
    
    
    mat = meshcat.geometry.MeshLambertMaterial(color=utils.rgb_to_hex((255,0,0)), wireframe=False)
    mat.opacity = 0.5
    vis["plane"][f"{geomA.get_value()}, {geomB.get_value()}"].set_object(
                meshcat.geometry.TriangularMeshGeometry(verts_tf, plane_triangles),
                mat)
    
    mat.opacity = 1.0
    utils.plot_point(loc = p1, radius = 0.05, mat = mat, vis = vis["plane"][f"{geomA.get_value()}, {geomB.get_value()}"], marker_id= 'p1')
    mat = meshcat.geometry.MeshLambertMaterial(color=utils.rgb_to_hex((255,255,0)), wireframe=False)
    mat.opacity = 1.0
    utils.plot_point(loc = p2, radius = 0.05, mat = mat, vis = vis["plane"][f"{geomA.get_value()}, {geomB.get_value()}"], marker_id= 'p2')
    
    mat.opacity = 1.0
    mat = meshcat.geometry.MeshLambertMaterial(color=utils.rgb_to_hex((0,255,0)), wireframe=False)
    utils.plot_point(loc = p1_other, radius = 0.05, mat = mat, vis = vis["plane"][f"{geomA.get_value()}, {geomB.get_value()}"], marker_id= 'p1_other')
    mat = meshcat.geometry.MeshLambertMaterial(color=utils.rgb_to_hex((0,0,255)), wireframe=False)
    mat.opacity = 1.0
    utils.plot_point(loc = p2_other, radius = 0.05, mat = mat, vis = vis["plane"][f"{geomA.get_value()}, {geomB.get_value()}"], marker_id= 'p2_other')


# cur_t = np.array([0.57,-0.73,0.37])
cur_t = np.array([-1.03,-.33,-.23])
showres_t(cur_t)

def plane_callback(t):
    for geomA, geomB in planes.keys():
        plot_plane_geom_id(geomA, geomB, planes, t)
        
plane_callback(cur_t)

{(<GeometryId value=71>, <GeometryId value=113>): (array([<Polynomial "4385.8171985897443*1 + 51.150586896893977*t[2] + 859.3986492011469*t[2]^2 + 507.41949431459938*t[1] + 759.4851375999848*t[1] * t[2] + 1972.4974983743755*t[1]^2 + -1989.072073883704*t[0] + -292.31830261111872*t[0] * t[2] + 3346.6525120426986*t[0] * t[1] + 1549.871676344947*t[0]^2">,
       <Polynomial "1.4038894540064596e-10*1 + 1.8701332953023992e-10*t[2] + -1.3442344491838083e-10*t[2]^2 + 1.7040513005635697e-11*t[1] + 1.4247504931700615e-10*t[1] * t[2] + -1.1249738336303408e-10*t[1]^2 + -8.5138390076624688e-10*t[0] + -1.3735207112335151e-10*t[0] * t[2] + 4.7424179538505531e-10*t[0] * t[1] + -3.0366401877054071e-10*t[0]^2">,
       <Polynomial "-1215.3512028938633*1 + -889.32134624088792*t[2] + 329.27087473724606*t[2]^2 + 3173.1532374752378*t[1] + 117.41947291222917*t[1] * t[2] + 1215.9906226397095*t[1]^2 + -3333.0347439795091*t[0] + 61.827418161867158*t[0] * t[2] + 66.20443431643254*t[0] * t[1] + 2494.5253353910252

In [51]:
def random_next_point_in_polytope(polytope, start_point, scale = 1):
    next_point = scale*np.random.randn(*start_point.shape)+start_point
    iter_num = 0
    max_iter = 100
    while not polytope.PointInSet(next_point) and iter_num < max_iter:
        next_point = scale*np.random.randn(*start_point.shape)+start_point
        iter_num +=1
    if iter_num >= max_iter:
        raise ValueError("failed to find next point")
    return next_point

def check_list_of_poly_are_pos_at_point(poly_list, t, t0):
    eval_dict = dict(zip(t, t0))
    failures = []
    for i, p in enumerate(poly_list):
        cond = p.Evaluate(eval_dict) > 0
        if not cond:
            failures.append((i, p, t0))
    all_pos = len(failures) < 1
    return all_pos, failures

def check_list_of_poly_are_pos_in_poltope(poly_list, polytope, t, num_points = 100):
    t0 = polytope.ChebyshevCenter()
    all_pos = True
    all_fails = []
    t_list = [t0]
    for i in range(num_points):
        all_pos_t0, failures = check_list_of_poly_are_pos_at_point(poly_list, t, t0)
#         print(f"All pos {all_pos_t0} for t = {t0}")
        if not all_pos_t0:
            all_fails += failures
            all_pos = False
        t0 = random_next_point_in_polytope(polytope, t0, scale = 0.1)
        t_list.append(t0)
    return all_pos, all_fails, np.array(t_list).T
        

In [52]:
pair_iter = iter(pairs)
good_pair = next(pair_iter)
prob_pair = next(pair_iter)
all_equal = True
evaled_pairs = []
vert_poly_evaled_list = []

def check_s_prod_pair_eq_constraint_satisfied(s_prod_pairs_polys, tol= 1e-5):
    all_equal = True
    failures = {}
    for pair, poly_list in s_prod_pairs_polys.items():
        for constraint_poly, vert_poly in poly_list:
            cur_equal = constraint_poly.CoefficientsAlmostEqual(vert_poly, tol)

            if not cur_equal:
                all_equal = False
                if pair in failures.keys():
                    failures[pair].append((constraint_poly, vert_poly))

                else:
                    failures[pair] = [(constraint_poly, vert_poly)]

    return all_equal, failures

def random_check_s_prod_pair_positive(s_prod_pairs_polys, num_eval = 100):
    all_pos = True
    failures = {}
    t_list = []
    for pair, poly_list in s_prod_pairs_polys.items():
        _, vert_poly_evaled_list = tuple(zip(*poly_list))
        cur_pos, cur_failures, cur_t_list = check_list_of_poly_are_pos_in_poltope(vert_poly_evaled_list, 
                                                                  poly_to_cert_safe, 
                                                                  t_kin, 
                                                                  num_points = num_eval)
        if not cur_pos:
            all_pos = False
            if pair in failures.keys():
                failures[pair] += cur_failures

            else:
                failures[pair] = cur_failures
                
        t_list.append(cur_t_list)
    return all_pos, failures, t_list
all_equal, failures_eq =  check_s_prod_pair_eq_constraint_satisfied(s_prod_pairs_polys)  
all_pos, failures_pos, t_list =  random_check_s_prod_pair_positive(s_prod_pairs_polys,1000) 
print(f"All equal {all_equal}")
print(f"All pos on set {all_pos}")
# for constraint_poly, vert_poly in s_prod_pairs[good_pair]:
#     print(constraint_poly.EqualTo(vert_poly))

All equal True
All pos on set True


In [66]:
id_val = (71,117)
t_test = np.array([0.37,-0.73,0.67])
eval_dict = dict(zip(sym.Variables(t_kin), t_test))
pair_iter = iter(pairs)
tmp = next(pair_iter)
while (tmp[0].get_value(), tmp[1].get_value()) != id_val:
    tmp = next(pair_iter)
s_prod_to_test = s_prod_pairs_polys[tmp]
print(f"t in poly to cert {poly_to_cert_unsafe.PointInSet(t_test)}")
for constraint_poly, vert_poly in s_prod_to_test:
    print(vert_poly.Evaluate(eval_dict) <= -1)
    print(constraint_poly.Evaluate(eval_dict) >= 1)
    print()

t in poly to cert True
True
False

True
False

True
False

True
False

True
False

True
False

True
False

True
False

False
False

False
True

False
True

False
False

False
False

False
True

False
True

False
False



# Comments with Hongkai
1.) To reduce poly size always get transforms relative to middle link in kinematic chain between two bodies (not worried)

2.) make the hyperplane an affine function of t. Should allow plane to rotate

3.) Do line search on Ct <= b- epsilon to scale regions

4.) Rational polynomials should have coordinate degree 2. Track down this bug ASAP
